# Hyperparameter Tuning using HyperDrive

In [1]:
#experiments management
from azureml.core import Workspace, Experiment,Dataset
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy,MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import GridParameterSampling
from azureml.train.hyperdrive import choice
import os

#compute resources management
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


## Dataset


In [2]:
#setting workspace from the Azure configuration file
ws = Workspace.from_config()
experiment_name = 'hyp-LR-experiment'

experiment=Experiment(ws, experiment_name)
dataset = Dataset.get_by_name(ws, name='heart_disease')
data = dataset.to_pandas_dataframe()
#(data.loc[:,data.columns!='target'], data['target'])

## Hyperdrive Configuration


In [3]:
#creating the compute target

#name of the compute cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already (good practice extracted by Microsoft official documentation: https://docs.microsoft.com/)
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
#creating compute (cpu) cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# Early termination for bad performance runs.This is not required if you are using Bayesian sampling.

early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

#define parameter search space -->random sampling over a hyperparameter search space (this would be good for first approach)
# dictionary{key:parameter, value: distribution}

param_sampling = RandomParameterSampling( {
        "--C": uniform(0.1, 1),
        "--max_iter": choice(25, 50, 75, 100)})


# Create a SKLearn estimator for use with train.py

estimator = SKLearn(source_directory='.',#folder for experiment configuration files.
              entry_script=  'train.py', #relative path to the file used to start training (training script)
              compute_target=cpu_cluster)

primary_metric_name='Accuracy'
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=estimator,
                                hyperparameter_sampling=param_sampling,
                                policy=early_termination_policy,
                                primary_metric_name=primary_metric_name,
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=100)
                                #max_total_runs=40)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

#submiting the exp
hyperdrive_run = experiment.submit(hyperdrive_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
#visualizing `progress of runs
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model


In [7]:
import joblib
# Getting the best model, saving and displaying all properties.

#finding the best model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']

print('Best Run Id: ', best_run.id)
print('\nAccuracy of best run :', best_run_metrics['Accuracy'])
print('\nHyperparam values of best run:',parameter_values['arguments'])


#saving the best model--> find the best one 
model = best_run.register_model(model_name='best_model_lr', model_path='outputs/model.joblib')

Best Run Id:  HD_c855f05a-ff44-4da8-a735-a927b663b45b_19

Accuracy of best run : 0.8852459016393442

Hyperparam values of best run: ['--C', '0.44459488726824914', '--max_iter', '100']


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service